In [4]:
# import requests
# import pandas as pd
# import time
# from datetime import datetime
# import json

# class RedditScraper:
#     def __init__(self):
#         self.headers = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#         }
        
#     def fetch_subreddit_posts(self, subreddit, limit=25):
#         """
#         Fetch posts from a subreddit using Reddit's JSON API
        
#         Args:
#             subreddit (str): Name of the subreddit without the 'r/'
#             limit (int): Maximum number of posts to fetch (default 25)
        
#         Returns:
#             pandas.DataFrame: DataFrame containing post data
#         """
#         posts_data = []
#         after = None
#         posts_collected = 0
        
#         while posts_collected < limit:
#             url = f'https://www.reddit.com/r/{subreddit}/hot.json?limit=100'
#             if after:
#                 url += f'&after={after}'
                
#             try:
#                 response = requests.get(url, headers=self.headers)
#                 response.raise_for_status()
#                 data = response.json()
                
#                 for post in data['data']['children']:
#                     post_data = post['data']
                    
#                     posts_data.append({
#                         'thread_id': post_data.get('id'),  # Unique identifier for the post
#                         'title': post_data.get('title'),
#                         'score': post_data.get('score'),
#                         'created_utc': datetime.fromtimestamp(post_data.get('created_utc', 0)),
#                         'num_comments': post_data.get('num_comments'),
#                         'url': post_data.get('url'),
#                         'selftext': post_data.get('selftext'),
#                         'author': post_data.get('author'),
#                         'upvote_ratio': post_data.get('upvote_ratio'),
#                         'is_video': post_data.get('is_video', False),
#                         'permalink': 'https://www.reddit.com' + post_data.get('permalink', '')
#                     })
                    
#                     posts_collected += 1
#                     if posts_collected >= limit:
#                         break
                
#                 after = data['data'].get('after')
#                 if not after:
#                     break
                    
#                 time.sleep(2)
                
#             except requests.exceptions.RequestException as e:
#                 print(f"Error fetching posts: {e}")
#                 break
                
#         return pd.DataFrame(posts_data)

#     def fetch_post_comments(self, post_id, permalink):
#         """
#         Fetch comments for a specific post
        
#         Args:
#             post_id (str): The ID of the post
#             permalink (str): The permalink of the post
        
#         Returns:
#             list: List of comment dictionaries
#         """
#         comments_data = []
        
#         try:
#             # Remove the domain from permalink and add .json
#             json_url = permalink.replace('https://www.reddit.com', '') + '.json'
#             url = f'https://www.reddit.com{json_url}'
            
#             response = requests.get(url, headers=self.headers)
#             response.raise_for_status()
#             data = response.json()
            
#             # Process comments recursively
#             def process_comments(comment_data, parent_id=None, level=0):
#                 if isinstance(comment_data, dict):
#                     if comment_data.get('kind') == 't1':  # t1 represents a comment
#                         comment = comment_data['data']
#                         comments_data.append({
#                             'thread_id': post_id,  # Link to the parent post
#                             'comment_id': comment.get('id'),
#                             'parent_comment_id': parent_id,  # Track comment parent
#                             'level': level,  # Track comment nesting level
#                             'author': comment.get('author'),
#                             'body': comment.get('body'),
#                             'score': comment.get('score'),
#                             'created_utc': datetime.fromtimestamp(comment.get('created_utc', 0))
#                         })
                        
#                         # Process replies
#                         replies = comment.get('replies')
#                         if replies and isinstance(replies, dict):
#                             for child in replies.get('data', {}).get('children', []):
#                                 process_comments(child, comment.get('id'), level + 1)
                    
#                     # Process children if present
#                     elif 'data' in comment_data and 'children' in comment_data['data']:
#                         for child in comment_data['data']['children']:
#                             process_comments(child, parent_id, level)
            
#             # Start processing from the comment tree
#             if len(data) > 1:  # Make sure we have comments data
#                 process_comments(data[1])  # data[0] is the post, data[1] is the comments
                
#             time.sleep(2)  # Be nice to Reddit's servers
            
#         except requests.exceptions.RequestException as e:
#             print(f"Error fetching comments for post {post_id}: {e}")
        
#         return comments_data

#     def scrape_subreddit(self, subreddit, post_limit=25, min_comments=5):
#         """
#         Scrape both posts and comments from a subreddit
        
#         Args:
#             subreddit (str): Subreddit name
#             post_limit (int): Number of posts to fetch
#             min_comments (int): Minimum number of comments for a post to be included
        
#         Returns:
#             tuple: (posts DataFrame, comments DataFrame)
#         """
#         print(f"Fetching posts from r/{subreddit}...")
#         posts_df = self.fetch_subreddit_posts(subreddit, limit=post_limit)
        
#         # Filter posts with minimum comments
#         posts_df = posts_df[posts_df['num_comments'] >= min_comments]
        
#         all_comments = []
#         total_posts = len(posts_df)
        
#         print(f"Fetching comments for {total_posts} posts...")
#         for idx, post in posts_df.iterrows():
#             print(f"Fetching comments for post {idx + 1}/{total_posts}")
#             comments = self.fetch_post_comments(post['thread_id'], post['permalink'])
#             all_comments.extend(comments)
        
#         comments_df = pd.DataFrame(all_comments)
        
#         return posts_df, comments_df

#     def save_data(self, posts_df, comments_df, subreddit):
#         """Save posts and comments to CSV and JSON formats"""
#         timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
#         # Save posts
#         posts_csv = f'{subreddit}_posts_{timestamp}.csv'
#         posts_json = f'{subreddit}_posts_{timestamp}.json'
#         posts_df.to_csv(posts_csv, index=False)
#         posts_df.to_json(posts_json, orient='records', date_format='iso')
        
#         # Save comments
#         comments_csv = f'{subreddit}_comments_{timestamp}.csv'
#         comments_json = f'{subreddit}_comments_{timestamp}.json'
#         comments_df.to_csv(comments_csv, index=False)
#         comments_df.to_json(comments_json, orient='records', date_format='iso')
        
#         print(f"\nData saved to:")
#         print(f"Posts: {posts_csv} and {posts_json}")
#         print(f"Comments: {comments_csv} and {comments_json}")


# scraper = RedditScraper()

# # Configuration
# SUBREDDIT = "python"
# POST_LIMIT = 10  # Number of posts to fetch
# MIN_COMMENTS = 5  # Only fetch posts with at least this many comments

# # Scrape data
# posts_df, comments_df = scraper.scrape_subreddit(
#     SUBREDDIT,
#     post_limit=POST_LIMIT,
#     min_comments=MIN_COMMENTS
# )

# # Display basic stats
# print(f"\nCollected {len(posts_df)} posts and {len(comments_df)} comments")

# # Show example of thread structure
# if not comments_df.empty:
#     print("\nExample comment thread structure:")
#     sample_thread = comments_df[comments_df['thread_id'] == comments_df['thread_id'].iloc[0]]

# posts_df.head(2)


Fetching posts from r/python...
Fetching comments for 6 posts...
Fetching comments for post 3/6
Fetching comments for post 4/6
Fetching comments for post 5/6
Fetching comments for post 7/6
Fetching comments for post 8/6
Fetching comments for post 10/6

Collected 6 posts and 285 comments

Example comment thread structure:


,thread_id,title,score,created_utc,num_comments,url,selftext,author,upvote_ratio,is_video,permalink
2,1gcq5rg,Configuration format,37,2024-10-26 10:24:37,49,https://www.reddit.com/r/Python/comments/1gcq5...,I currently use JSONs for storing my configura...,Messmer_Impaler,0.89,False,https://www.reddit.com/r/Python/comments/1gcq5...
3,1gcob89,Effortlessly Monitor and Manage Concurrent Fun...,21,2024-10-26 09:00:56,7,https://www.reddit.com/r/Python/comments/1gcob...,**What my project does**\n\n`functionmonitor` ...,Vivid-Job2568,0.88,False,https://www.reddit.com/r/Python/comments/1gcob...


In [5]:
# comments_df.head(2)

,thread_id,comment_id,parent_comment_id,level,author,body,score,created_utc
0,1gcq5rg,ltvpp2i,None,0,swigganicks,"I must ask, what kind of configuration file is...",80,2024-10-26 10:34:05
1,1gcq5rg,ltvqi6w,ltvpp2i,1,Messmer_Impaler,There's a repeating pattern to the configs whi...,1,2024-10-26 10:38:24


In [11]:
# import requests
# import pandas as pd
# import time
# from datetime import datetime
# import json

# class SubredditScraper:
#     def __init__(self):
#         self.headers = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#         }

#     def get_subreddit_info(self, subreddit):
#         """
#         Fetch detailed information about a specific subreddit
        
#         Args:
#             subreddit (str): Subreddit name without 'r/'
        
#         Returns:
#             dict: Dictionary containing subreddit information
#         """
#         url = f'https://www.reddit.com/r/{subreddit}/about.json'
        
#         try:
#             response = requests.get(url, headers=self.headers)
#             response.raise_for_status()
#             data = response.json()['data']
            
#             # Extract all available attributes
#             subreddit_info = {
#                 'display_name': data.get('display_name'),
#                 'title': data.get('title'),
#                 'display_name_prefixed': data.get('display_name_prefixed'),
#                 'subscribers': data.get('subscribers'),
#                 'active_users': data.get('active_user_count'),
#                 'created_utc': datetime.fromtimestamp(data.get('created_utc', 0)),
#                 'description': data.get('description'),
#                 'public_description': data.get('public_description'),
#                 'subreddit_type': data.get('subreddit_type'),
#                 'over18': data.get('over18'),
#                 'is_default': data.get('default_set', False),
#                 'allows_images': data.get('allows_images', False),
#                 'allows_videos': data.get('allows_videos', False),
#                 'allows_polls': data.get('allows_polls', False),
#                 'allow_galleries': data.get('allow_galleries', False),
#                 'allow_predictions': data.get('allow_predictions', False),
#                 'spoilers_enabled': data.get('spoilers_enabled', False),
#                 'comment_score_hide_mins': data.get('comment_score_hide_mins'),
#                 'submission_type': data.get('submission_type'),
#                 'restrict_posting': data.get('restrict_posting', False),
#                 'restrict_commenting': data.get('restrict_commenting', False),
#                 'lang': data.get('lang'),
#                 'url': f"https://www.reddit.com/r/{subreddit}/",
#                 'wiki_enabled': data.get('wiki_enabled', False),
#                 'is_crosspostable': data.get('is_crosspostable', False),
#                 'quarantine': data.get('quarantine', False),
#                 'community_icon': data.get('community_icon'),
#                 'banner_img': data.get('banner_img'),
#                 'emojis_enabled': data.get('emojis_enabled', False),
#                 'advertiser_category': data.get('advertiser_category'),
#                 'public_traffic': data.get('public_traffic', False),
#                 'subscribers_rank': data.get('subscribers_rank'),
#                 'video_stream_enabled': data.get('video_stream_enabled', False),
#                 'header_title': data.get('header_title'),
#                 'can_assign_link_flair': data.get('can_assign_link_flair', False),
#                 'can_assign_user_flair': data.get('can_assign_user_flair', False),
#                 'user_flair_enabled': data.get('user_flair_enabled_in_sr', False),
#                 'link_flair_enabled': data.get('link_flair_enabled', False),
#                 'primary_color': data.get('primary_color'),
#                 'key_color': data.get('key_color'),
#                 'last_checked': datetime.now()
#             }
            
#             return subreddit_info
            
#         except requests.exceptions.RequestException as e:
#             print(f"Error fetching data for r/{subreddit}: {e}")
#             return None

#     def get_multiple_subreddits(self, subreddits, save_results=True):
#         """
#         Fetch information for multiple subreddits
        
#         Args:
#             subreddits (list): List of subreddit names
#             save_results (bool): Whether to save results to files
        
#         Returns:
#             pandas.DataFrame: DataFrame containing all subreddit information
#         """
#         all_data = []
#         total = len(subreddits)
        
#         for idx, subreddit in enumerate(subreddits, 1):
#             print(f"Fetching data for r/{subreddit} ({idx}/{total})")
            
#             subreddit_data = self.get_subreddit_info(subreddit)
#             if subreddit_data:
#                 all_data.append(subreddit_data)
            
#             # Be nice to Reddit's servers
#             time.sleep(2)
        
#         # Create DataFrame
#         df = pd.DataFrame(all_data)
        
#         if save_results and not df.empty:
#             self.save_data(df)
        
#         return df

#     def save_data(self, df):
#         """Save the subreddit information to both CSV and JSON formats"""
#         timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
#         # Save as CSV
#         csv_filename = f'subreddit_info_{timestamp}.csv'
#         df.to_csv(csv_filename, index=False)
        
#         # Save as JSON with better formatting
#         json_filename = f'subreddit_info_{timestamp}.json'
#         df.to_json(json_filename, orient='records', date_format='iso', indent=4)
        
#         print(f"\nData saved to:")
#         print(f"CSV: {csv_filename}")
#         print(f"JSON: {json_filename}")

#     def print_summary(self, df):
#         """Print summary statistics about the collected subreddit data"""
#         if df.empty:
#             print("No data collected!")
#             return
            
#         print("\nSummary Statistics:")
#         print(f"Total subreddits analyzed: {len(df)}")
        
#         print("\nSubscriber Statistics:")
#         print(df['subscribers'].describe())
        
#         print("\nActive Users Statistics:")
#         print(df['active_users'].describe())
        
#         print("\nAge of Subreddits:")
#         df['age_days'] = (datetime.now() - df['created_utc']).dt.days
#         print(df['age_days'].describe())
        
#         print("\nSubreddit Types Distribution:")
#         print(df['subreddit_type'].value_counts())
        
#         print("\nFeature Availability:")
#         features = ['allows_images', 'allows_videos', 'allows_polls', 
#                    'allow_galleries', 'wiki_enabled', 'emojis_enabled']
#         for feature in features:
#             if feature in df.columns:
#                 print(f"{feature}: {df[feature].mean()*100:.1f}% enabled")

# # Example usage
# if __name__ == "__main__":
#     scraper = SubredditScraper()
    
#     # Example list of subreddits to analyze
#     subreddits_to_analyze = [
#         "python",
#         "programming",
#         "datascience",
#         "machinelearning",
#         "javascript",
#         "webdev",
#         "technology",
#         "science"
#     ]
    
#     # Get information for all subreddits
#     subreddits_df = scraper.get_multiple_subreddits(subreddits_to_analyze)
    
#     # Print summary statistics
#     scraper.print_summary(subreddits_df)
    
#     # Example: Get top 5 subreddits by subscriber count
#     top_by_subscribers = subreddits_df.nlargest(5, 'subscribers')
#     print("\nTop 5 Subreddits by Subscribers:")
#     print(top_by_subscribers[['display_name', 'subscribers', 'active_users']])
    
#     # Example: Get newest subreddits
#     newest_subreddits = subreddits_df.nsmallest(5, 'created_utc')
#     print("\nNewest Subreddits:")
#     print(newest_subreddits[['display_name', 'created_utc']])

Fetching data for r/python (1/8)
Fetching data for r/programming (2/8)
Fetching data for r/datascience (3/8)
Fetching data for r/machinelearning (4/8)
Fetching data for r/javascript (5/8)
Fetching data for r/webdev (6/8)
Fetching data for r/technology (7/8)
Fetching data for r/science (8/8)

Data saved to:
CSV: subreddit_info_20241026_174616.csv
JSON: subreddit_info_20241026_174616.json

Summary Statistics:
Total subreddits analyzed: 8

Subscriber Statistics:
count    8.000000e+00
mean     8.549066e+06
std      1.118129e+07
min      1.286093e+06
25%      2.363017e+06
50%      2.788873e+06
75%      9.241988e+06
max      3.299837e+07
Name: subscribers, dtype: float64

Active Users Statistics:
count       8.000000
mean      790.375000
std      1074.701412
min        95.000000
25%       125.750000
50%       173.000000
75%      1054.000000
max      2747.000000
Name: active_users, dtype: float64

Age of Subreddits:
count       8.000000
mean     5988.250000
std       616.676055
min      4830.

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime

class SubredditListScraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.collected_subreddits = set()

    def fetch_subreddits(self, limit=1000, min_subscribers=1000):
        """
        Fetch subreddits from Reddit
        
        Args:
            limit (int): Number of subreddits to fetch
            min_subscribers (int): Minimum number of subscribers required
        """
        subreddits_data = []
        after = None
        
        while len(subreddits_data) < limit:
            try:
                url = f'https://www.reddit.com/subreddits/popular.json?limit=100'
                if after:
                    url += f'&after={after}'
                
                print(f"Fetching subreddits... Current count: {len(subreddits_data)}")
                response = requests.get(url, headers=self.headers)
                response.raise_for_status()
                data = response.json()
                
                for subreddit in data['data']['children']:
                    subreddit_data = subreddit['data']
                    
                    # Skip if already collected
                    if subreddit_data['display_name'] in self.collected_subreddits:
                        continue
                        
                    # Check subscriber count
                    if subreddit_data.get('subscribers', 0) >= min_subscribers:
                        sub_info = {
                            'display_name': subreddit_data.get('display_name'),
                            'display_name_prefixed': subreddit_data.get('display_name_prefixed'),
                            'title': subreddit_data.get('title'),
                            'subscribers': subreddit_data.get('subscribers', 0),
                            'active_users': subreddit_data.get('active_user_count', 0),
                            'created_utc': datetime.fromtimestamp(subreddit_data.get('created_utc', 0)),
                            'description': subreddit_data.get('description'),
                            'subreddit_type': subreddit_data.get('subreddit_type'),
                            'over18': subreddit_data.get('over18', False),
                            'url': f"https://www.reddit.com/r/{subreddit_data.get('display_name')}/"
                        }
                        
                        subreddits_data.append(sub_info)
                        self.collected_subreddits.add(subreddit_data['display_name'])
                
                # Get pagination token
                after = data['data'].get('after')
                if not after:
                    print("Reached end of listings")
                    break
                    
                time.sleep(2)  # Be nice to Reddit's servers
                
            except requests.exceptions.RequestException as e:
                print(f"Error fetching data: {e}")
                break
                
        return pd.DataFrame(subreddits_data)

    def save_data(self, df):
        """Save the subreddit list to both CSV and JSON formats"""
        if df.empty:
            print("No data to save!")
            return
            
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Save as CSV
        csv_filename = f'subreddit_list_{timestamp}.csv'
        df.to_csv(csv_filename, index=False)
        
        print(f"\nData saved to: {csv_filename}")

    def print_summary(self, df):
        """Print summary statistics about the collected subreddits"""
        if df.empty:
            print("No data collected!")
            return
            
        print("\nSummary Statistics:")
        print(f"Total subreddits collected: {len(df)}")
        print("\nSubscriber Statistics:")
        print(df['subscribers'].describe())
        
        # Print top 10 subreddits
        print("\nTop 10 Subreddits:")
        top_10 = df.nlargest(10, 'subscribers')
        print(top_10[['display_name_prefixed', 'title', 'subscribers', 'active_users']])

# Example usage
if __name__ == "__main__":
    scraper = SubredditListScraper()
    
    # Collect subreddits with at least 10,000 subscribers
    subreddits_df = scraper.fetch_subreddits(
        limit=10000,  # Try to collect 1000 subreddits
        min_subscribers=10000  # Only subreddits with 10k+ subscribers
    )
    
    # Print summary and save
    scraper.print_summary(subreddits_df)
    scraper.save_data(subreddits_df)

Fetching subreddits... Current count: 0
Fetching subreddits... Current count: 100
Fetching subreddits... Current count: 200
Fetching subreddits... Current count: 300
Fetching subreddits... Current count: 400
Fetching subreddits... Current count: 500
Fetching subreddits... Current count: 600
Fetching subreddits... Current count: 700
Fetching subreddits... Current count: 800
Fetching subreddits... Current count: 900
Fetching subreddits... Current count: 1000
Fetching subreddits... Current count: 1100
Fetching subreddits... Current count: 1200
Fetching subreddits... Current count: 1300
Fetching subreddits... Current count: 1400
Fetching subreddits... Current count: 1500
Fetching subreddits... Current count: 1600
Fetching subreddits... Current count: 1700
Fetching subreddits... Current count: 1800
Fetching subreddits... Current count: 1900
Fetching subreddits... Current count: 2000
Fetching subreddits... Current count: 2099
Fetching subreddits... Current count: 2199
Fetching subreddits... 

In [4]:
import requests
import pandas as pd
import time
from datetime import datetime
import json

class MultiRedditScraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def fetch_top_posts(self, subreddit, limit=10):
        """
        Fetch top posts from a subreddit using Reddit's JSON API
        
        Args:
            subreddit (str): Name of the subreddit without the 'r/'
            limit (int): Maximum number of posts to fetch
            
        Returns:
            list: List of post dictionaries
        """
        posts_data = []
        url = f'https://www.reddit.com/r/{subreddit}/top.json?t=all&limit={limit}'
        
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            data = response.json()
            
            for post in data['data']['children']:
                post_data = post['data']
                posts_data.append({
                    'subreddit': subreddit,
                    'title': post_data.get('title'),
                    'score': post_data.get('score'),
                    'num_comments': post_data.get('num_comments'),
                    'created_utc': datetime.fromtimestamp(post_data.get('created_utc', 0)),
                    'url': post_data.get('url'),
                    'permalink': 'https://www.reddit.com' + post_data.get('permalink', '')
                })
            
            time.sleep(2)  # Be nice to Reddit's servers
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching posts from r/{subreddit}: {e}")
        
        return posts_data

    def get_top_subreddits(self, subreddits_df, top_n=None, order='top'):
        """
        Filter subreddits DataFrame to get top or bottom N subreddits by subscriber count
        
        Args:
            subreddits_df (pandas.DataFrame): DataFrame containing subreddit information
            top_n (int): Number of subreddits to return. If None, returns all subreddits
            order (str): 'top' for most popular or 'bottom' for least popular subreddits
                
        Returns:
            pandas.DataFrame: Filtered DataFrame with selected subreddits
        """
        if top_n is None or top_n >= len(subreddits_df):
            return subreddits_df
        
        # Sort by subscribers
        sorted_df = subreddits_df.sort_values('subscribers', 
                                            ascending=True if order == 'bottom' else False)
        
        # Get top or bottom N
        return sorted_df.head(top_n).copy()

    def scrape_multiple_subreddits(self, subreddits_df, posts_per_subreddit=10, top_n=None, order='top'):
        """
        Scrape top posts from multiple subreddits
        
        Args:
            subreddits_df (pandas.DataFrame): DataFrame containing subreddit information
            posts_per_subreddit (int): Number of top posts to fetch per subreddit
            top_n (int): Optional number of subreddits to scrape (by subscriber count)
            order (str): 'top' for most popular or 'bottom' for least popular subreddits
                
        Returns:
            pandas.DataFrame: Combined DataFrame of all top posts
        """
        # Filter for selected subreddits
        filtered_df = self.get_top_subreddits(subreddits_df, top_n, order)
        total_subreddits = len(filtered_df)
        
        print(f"\nStarting to scrape {total_subreddits} subreddits...")
        if top_n:
            print(f"Filtering for {'top' if order == 'top' else 'bottom'} {top_n} subreddits by subscriber count")
            print("\nSelected subreddits:")
            for _, row in filtered_df.iterrows():
                print(f"r/{row['display_name']}: {row['subscribers']:,} subscribers")
        all_posts = []
        
        for idx, (_, row) in enumerate(filtered_df.iterrows(), 1):
            subreddit = row['display_name']
            print(f"\nProcessing {idx}/{total_subreddits}: r/{subreddit}")
            
            try:
                posts = self.fetch_top_posts(subreddit, limit=posts_per_subreddit)
                all_posts.extend(posts)
                print(f"✓ Successfully collected {len(posts)} posts")
                
            except Exception as e:
                print(f"✗ Error processing r/{subreddit}: {e}")
                continue
        
        # Convert to DataFrame
        if all_posts:
            posts_df = pd.DataFrame(all_posts)
            
            # Sort by score and num_comments
            posts_df = posts_df.sort_values(['subreddit', 'score', 'num_comments'], 
                                          ascending=[True, False, False])
            
            print(f"\nSuccessfully collected {len(posts_df)} posts from {posts_df['subreddit'].nunique()} subreddits")
        else:
            posts_df = pd.DataFrame(columns=['subreddit', 'title', 'score', 'num_comments', 
                                           'created_utc', 'url', 'permalink'])
            print("\nNo posts were collected")
        
        return posts_df

    def save_results(self, posts_df, filename_prefix='top_posts'):
        """Save results to CSV and JSON formats"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Save to CSV
        csv_filename = f'{filename_prefix}_{timestamp}.csv'
        posts_df.to_csv(csv_filename, index=False)
        
        # Save to JSON
        json_filename = f'{filename_prefix}_{timestamp}.json'
        posts_df.to_json(json_filename, orient='records', date_format='iso')
        
        print(f"\nData saved to:")
        print(f"CSV: {csv_filename}")
        print(f"JSON: {json_filename}")
        
        return csv_filename, json_filename

# Run the scraper
scraper = MultiRedditScraper()

# # For most popular subreddits
# top_posts_df = scraper.scrape_multiple_subreddits(
#     subreddits_df,
#     posts_per_subreddit=10,
#     top_n=5  # Gets 5 most popular subreddits
# )

# For least popular subreddits
bottom_posts_df = scraper.scrape_multiple_subreddits(
    subreddits_df,
    posts_per_subreddit=3,
    top_n=20,
    order='bottom'  # Gets 5 least popular subreddits
)

bottom_posts_df.tail()


Starting to scrape 20 subreddits...
Filtering for bottom 20 subreddits by subscriber count

Selected subreddits:
r/BetterDiscord: 10,069 subscribers
r/DanhengMains: 10,096 subscribers
r/NintendoSwitchBoxArt: 10,310 subscribers
r/DownSouth: 10,570 subscribers
r/DS4Windows: 10,895 subscribers
r/ChioriMains: 10,928 subscribers
r/CurseForge: 10,967 subscribers
r/insomniacleaks: 11,193 subscribers
r/FuckTAA: 11,343 subscribers
r/japanpeopletwitter: 11,473 subscribers
r/otvandfriendsrumors: 11,513 subscribers
r/opus_magnum: 11,585 subscribers
r/prime: 11,794 subscribers
r/JingYuanMains: 12,447 subscribers
r/dewrim: 12,858 subscribers
r/byu: 12,887 subscribers
r/bahn: 13,043 subscribers
r/BlackSwanMains_HSR: 13,058 subscribers
r/hospitalist: 13,093 subscribers
r/handbrake: 13,188 subscribers

Processing 1/20: r/BetterDiscord
✓ Successfully collected 3 posts

Processing 2/20: r/DanhengMains
✓ Successfully collected 3 posts

Processing 3/20: r/NintendoSwitchBoxArt
✓ Successfully collected 3 po

,subreddit,title,score,num_comments,created_utc,url,permalink
31,otvandfriendsrumors,Fuslie Confirms She's The Reason For NoahJ's D...,1170,365,2024-08-07 21:26:59,https://www.reddit.com/r/otvandfriendsrumors/c...,https://www.reddit.com/r/otvandfriendsrumors/c...
32,otvandfriendsrumors,Aaandddd she's back on Twitch,896,63,2024-07-12 18:03:41,https://i.redd.it/nec0jvnvq6cd1.png,https://www.reddit.com/r/otvandfriendsrumors/c...
36,prime,Nobody wants to drink this shit,3502,779,2023-09-10 08:16:00,https://i.redd.it/r2x9ibd33gnb1.jpg,https://www.reddit.com/r/prime/comments/16f2s9...
37,prime,Oops 🤫,2836,166,2024-02-09 08:36:54,https://i.redd.it/8qhf5cd38lhc1.jpeg,https://www.reddit.com/r/prime/comments/1amsa9...
38,prime,Nobody is buying this shit 😭,2756,505,2024-01-16 04:43:10,https://i.redd.it/hahidejisscc1.jpeg,https://www.reddit.com/r/prime/comments/19821s...


In [5]:
import requests
import pandas as pd
from datetime import datetime
import time

class RedditCommentScraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def prepare_posts_df(self, posts_df, top_n_posts=None, sort_by=['score', 'num_comments'], 
                        ascending=False, min_score=None, min_comments=None):
        """
        Sort and filter the posts dataframe before fetching comments
        """
        df = posts_df.copy()
        
        if min_score is not None:
            df = df[df['score'] >= min_score]
        if min_comments is not None:
            df = df[df['num_comments'] >= min_comments]
        
        df = df.sort_values(by=sort_by, ascending=ascending)
        df = df.drop_duplicates(subset=['title'])
        
        if top_n_posts is not None:
            df = df.head(top_n_posts)
        
        return df

    def extract_comment_data(self, comment, post_info):
        """Extract relevant data from a comment"""
        return {
            'subreddit': post_info['subreddit'],
            'post_title': post_info['title'],
            'post_score': post_info['score'],
            'post_created_utc': post_info['created_utc'],
            'comment_id': comment['data'].get('id'),
            'comment_author': comment['data'].get('author'),
            'comment_body': comment['data'].get('body'),
            'comment_score': comment['data'].get('score', 0),
            'comment_created_utc': datetime.fromtimestamp(comment['data'].get('created_utc', 0)),
            'post_url': post_info['url'],
            'comment_url': f"https://www.reddit.com{post_info['permalink']}{comment['data'].get('id')}",
        }
    
    def fetch_top_comments(self, post_df, num_comments=10):
        """
        Fetch top comments for each post in the dataframe, sorted by upvotes
        """
        all_comments = []
        total_posts = len(post_df)
        
        print(f"\nFetching top {num_comments} most upvoted comments for {total_posts} posts...")
        
        for idx, post in post_df.iterrows():
            print(f"\nProcessing post {idx + 1}/{total_posts}")
            print(f"Title: {post['title'][:100]}...")
            print(f"Post Score: {post['score']}, Number of Comments: {post['num_comments']}")
            
            try:
                json_url = post['permalink'].replace('https://www.reddit.com', '') + '.json'
                url = f'https://www.reddit.com{json_url}'
                
                response = requests.get(url, headers=self.headers)
                response.raise_for_status()
                data = response.json()
                
                if len(data) > 1:
                    comments_data = data[1]['data']['children']
                    
                    # Filter out non-comment entries and extract scores
                    valid_comments = [
                        comment for comment in comments_data 
                        if comment['kind'] == 't1' and comment['data'].get('score') is not None
                    ]
                    
                    # Sort comments by score (upvotes) in descending order
                    sorted_comments = sorted(
                        valid_comments,
                        key=lambda x: x['data'].get('score', 0),
                        reverse=True
                    )
                    
                    # Take only the top N comments
                    top_comments = sorted_comments[:num_comments]
                    
                    # Print comment scores for verification
                    print("\nTop comment scores for this post:")
                    for i, comment in enumerate(top_comments, 1):
                        score = comment['data'].get('score', 0)
                        print(f"Comment {i}: {score} upvotes")
                    
                    # Add to main list
                    for comment in top_comments:
                        all_comments.append(self.extract_comment_data(comment, post))
                
                time.sleep(2)
                
            except requests.exceptions.RequestException as e:
                print(f"Error fetching comments for post {idx + 1}: {e}")
                continue
                
        # Create DataFrame and sort
        comments_df = pd.DataFrame(all_comments)
        
        if not comments_df.empty:
            # Verify sorting by showing top comments for each post
            print("\nVerification of comment sorting:")
            for post_title in comments_df['post_title'].unique():
                post_comments = comments_df[comments_df['post_title'] == post_title]
                print(f"\nPost: {post_title[:100]}...")
                print("Comment scores:", post_comments['comment_score'].tolist())
        
        return comments_df
    
    def save_comments(self, comments_df, filename_prefix='reddit_comments'):
        """Save comments and print detailed statistics"""
        if comments_df.empty:
            print("No comments to save!")
            return
            
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Save files
        csv_filename = f'{filename_prefix}_{timestamp}.csv'
        json_filename = f'{filename_prefix}_{timestamp}.json'
        comments_df.to_csv(csv_filename, index=False)
        comments_df.to_json(json_filename, orient='records', date_format='iso')
        
        # Print detailed statistics
        print("\n=== Detailed Statistics ===")
        print(f"Files saved to:\n- CSV: {csv_filename}\n- JSON: {json_filename}")
        print(f"\nTotal unique posts: {comments_df['post_title'].nunique()}")
        print(f"Total comments collected: {len(comments_df)}")
        print(f"Average comments per post: {len(comments_df) / comments_df['post_title'].nunique():.1f}")
        
        # Print comment score statistics
        print("\nComment Score Statistics:")
        print(f"Maximum comment score: {comments_df['comment_score'].max()}")
        print(f"Average comment score: {comments_df['comment_score'].mean():.1f}")
        print(f"Minimum comment score: {comments_df['comment_score'].min()}")
        
        # Show distribution of comments per post
        print("\nComments per post:")
        comments_per_post = comments_df.groupby('post_title').agg({
            'comment_score': ['count', 'max', 'mean', 'min']
        }).round(1)
        comments_per_post.columns = ['num_comments', 'max_score', 'avg_score', 'min_score']
        print(comments_per_post)

def get_top_comments(posts_df, num_comments=10, top_n_posts=None, 
                    sort_by=['score', 'num_comments'], ascending=False,
                    min_score=None, min_comments=None, save_output=True):
    """
    Get top comments for posts, sorted by upvotes
    """
    scraper = RedditCommentScraper()
    
    filtered_posts = scraper.prepare_posts_df(
        posts_df,
        top_n_posts=top_n_posts,
        sort_by=sort_by,
        ascending=ascending,
        min_score=min_score,
        min_comments=min_comments
    )
    
    comments_df = scraper.fetch_top_comments(filtered_posts, num_comments)
    
    if save_output:
        scraper.save_comments(comments_df)
    
    return comments_df


# Get top 10 most upvoted comments for each of the top 5 highest-scoring posts
top_comments_df = get_top_comments(
    bottom_posts_df,
    num_comments=3,         # Get top 10 comments per post
    top_n_posts=10,          # Process only top 5 posts
    sort_by='score',        # Sort posts by score
    ascending=False,        # Sort in descending order
    min_score=100,         # Only posts with score >= 100
    min_comments=5         # Only posts with >= 5 comments
)
top_comments_df.head(3)


Fetching top 3 most upvoted comments for 10 posts...

Processing post 52/10
Title: Black Swan x Shorekeeper [art by McDoboArt]...
Post Score: 4614, Number of Comments: 44

Top comment scores for this post:
Comment 1: 68 upvotes
Comment 2: 55 upvotes
Comment 3: 14 upvotes

Processing post 49/10
Title: Einmal passiert, seit dem buche ich niemals die letzten Züge. Egal wie günstig sie sind....
Post Score: 3868, Number of Comments: 83

Top comment scores for this post:
Comment 1: 134 upvotes
Comment 2: 35 upvotes
Comment 3: 25 upvotes

Processing post 37/10
Title: Nobody wants to drink this shit...
Post Score: 3502, Number of Comments: 779

Top comment scores for this post:
Comment 1: 46 upvotes
Comment 2: 14 upvotes
Comment 3: 8 upvotes

Processing post 53/10
Title: You didn't need to be so direct, Black Swan....
Post Score: 3160, Number of Comments: 42

Top comment scores for this post:
Comment 1: 103 upvotes
Comment 2: 70 upvotes
Comment 3: 47 upvotes

Processing post 16/10
Title: Chio

,subreddit,post_title,post_score,post_created_utc,comment_id,comment_author,comment_body,comment_score,comment_created_utc,post_url,comment_url
0,BlackSwanMains_HSR,Black Swan x Shorekeeper [art by McDoboArt],4614,2024-10-15 06:02:53,ls197id,nisan_23,https://preview.redd.it/2vlgatn48xud1.png?widt...,68,2024-10-15 06:21:04,https://i.redd.it/cakdft9f4xud1.jpeg,https://www.reddit.comhttps://www.reddit.com/r...
1,BlackSwanMains_HSR,Black Swan x Shorekeeper [art by McDoboArt],4614,2024-10-15 06:02:53,ls1dtm1,Ahrrivederci,Black Swan has two strong arguments. But I lov...,55,2024-10-15 06:49:54,https://i.redd.it/cakdft9f4xud1.jpeg,https://www.reddit.comhttps://www.reddit.com/r...
2,BlackSwanMains_HSR,Black Swan x Shorekeeper [art by McDoboArt],4614,2024-10-15 06:02:53,ls1r5l6,HexagonII,Symmetrical docking,14,2024-10-15 08:05:29,https://i.redd.it/cakdft9f4xud1.jpeg,https://www.reddit.comhttps://www.reddit.com/r...


In [53]:
def aggregate_post_comments(df):
    """
    Aggregate post titles and comments into a list of strings,
    one element per post containing the title and all its comments.
    
    Args:
        df (pandas.DataFrame): DataFrame containing post_title and comment_body
        
    Returns:
        list: List of strings, each containing a post's title and its comments
    """
    aggregated_texts = []
    
    # Group by post_title
    grouped = df.groupby('post_title')
    
    for title, group in grouped:
        # Start with the post title
        post_text = f"{title}\n\n"
        
        # Add each comment with its score
        comments = group.sort_values('comment_score', ascending=False)
        
        for idx, row in comments.iterrows():
            comment_text = row['comment_body']
            comment_score = row['comment_score']
            if pd.notna(comment_text):  # Check if comment is not NaN
                post_text += f"COMMENT ({comment_score} upvotes): {comment_text}\n\n"
        
        aggregated_texts.append(post_text)
    
    return aggregated_texts

# Example usage:
aggregated_posts = aggregate_post_comments(top_comments_df)

# Print the first post as an example
if aggregated_posts:
    print("Example of first aggregated post:")
    print("="*80)
    print(aggregated_posts[0])
    print("\nTotal number of posts:", len(aggregated_posts))

Example of first aggregated post:
30 years of "freedom" in South Africa 🇿🇦 - and locals are fighting for water in the street. The ANC has failed... yet the people will still vote for them.

COMMENT (56 upvotes): Its clearly the white persons fault. Don't know where he is but when we find him he's got a lot of explaining to do.

COMMENT (49 upvotes): Such an apt metaphor of South Africa. Fighting over limited resources, and wasting them in the process

COMMENT (31 upvotes): It's truly disappointing when a community can't get together to get through a water crisis. 

Greed and selfishness is a disease that thrives in our country at the moment, both on the streets and in government.



Total number of posts: 42


In [54]:
aggregated_posts

['30 years of "freedom" in South Africa 🇿🇦 - and locals are fighting for water in the street. The ANC has failed... yet the people will still vote for them.\n\nCOMMENT (56 upvotes): Its clearly the white persons fault. Don\'t know where he is but when we find him he\'s got a lot of explaining to do.\n\nCOMMENT (49 upvotes): Such an apt metaphor of South Africa. Fighting over limited resources, and wasting them in the process\n\nCOMMENT (31 upvotes): It\'s truly disappointing when a community can\'t get together to get through a water crisis. \n\nGreed and selfishness is a disease that thrives in our country at the moment, both on the streets and in government.\n\n',
 "Animal Crossing: New Horizons Slipcover\n\nCOMMENT (7 upvotes): Something I put together in anticipation. Never printed a slipcover, so I can't answer any of those kinds of questions.\n\nCOMMENT (3 upvotes): I wonder what inspired you :P\nLooks great\n\nCOMMENT (1 upvotes): Do you have the original cover\n\n",
 "Announcem

In [65]:
import asyncio
from datetime import datetime, timedelta
import json
from typing import List, Dict
import nest_asyncio
from openai import AsyncAzureOpenAI
import os
from dotenv import load_dotenv
import pandas as pd
import signal
import sys

# Enable nested event loops for Jupyter
nest_asyncio.apply()

class GracefulExit(Exception):
    pass

def signal_handler(signum, frame):
    raise GracefulExit()

signal.signal(signal.SIGINT, signal_handler)

class ResultSaver:
    def __init__(self):
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        self.filename = f"business_ideas_{self.timestamp}.csv"
        self.results = []
        
    def add_result(self, content: str, ideas: List[Dict]):
        for idea in ideas:
            row = {
                'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'original_content': content[:500],  # Truncate long content
                'title': idea.get('title', ''),
                'business': idea.get('business', ''),
                'automation': json.dumps(idea.get('automation', [])),
                'notes': json.dumps(idea.get('notes', {}))
            }
            self.results.append(row)
        self.save_progress()
    
    def save_progress(self):
        df = pd.DataFrame(self.results)
        df.to_csv(self.filename, index=False)
        print(f"Progress saved to {self.filename}")

class BusinessIdeaGenerator:
    def __init__(self, 
                 batch_size: int = 5,
                 topic: str = "web browsing automation",
                 custom_prompt: str = None):
        self.batch_size = batch_size
        self.rate_limiter = RateLimiter(calls_per_minute=60)
        self.client = initialize_azure_openai_client()
        self.topic = topic
        self.custom_prompt = custom_prompt or self._get_default_prompt()
        self.result_saver = ResultSaver()

    def _get_default_prompt(self) -> str:
        return """You are a business consultant specializing in automated SaaS solutions. 
        For each Reddit post/comment provided, generate B2B SaaS business ideas that leverage web automation.
        Each idea must:
        1. Use web automation (Playwright/Selenium) to perform tasks without human intervention
        2. Use AI agents for decision making
        3. Be purely digital with no physical labor
        4. Have a clear subscription-based revenue model

        Your response must be a valid JSON array. Format each idea exactly as:
        [
            {
                "title": "The original Reddit post title",
                "business": "Concise explanation of what service the business provides",
                "automation": [
                    "Step 1 of the automation process",
                    "Step 2 of the automation process",
                    "Additional steps..."
                ],
                "notes": {
                    "monetization": "How the business makes money (pricing tiers, subscription model)",
                    "industry": "Specific industry vertical this serves",
                    "market": {
                        "consumer": "Whether/how it serves individual consumers",
                        "prosumer": "Whether/how it serves professional consumers",
                        "enterprise": "Whether/how it serves enterprise clients"
                    }
                }
            }
        ]

        If no viable ideas can be generated, return an empty array: []
        Focus on practical, implementable ideas using current technology.
        Each automation step should be specific and technical, explaining exactly how web automation would accomplish the task."""

    async def generate_ideas(self, content_list: List[str]) -> List[Dict]:
        all_ideas = []
        try:
            for i in range(0, len(content_list), self.batch_size):
                batch = content_list[i:i + self.batch_size]
                await self.rate_limiter.acquire()
                batch_ideas = await self._process_batch(batch)
                all_ideas.extend(batch_ideas)
        except GracefulExit:
            print("\nGracefully shutting down and saving progress...")
        except Exception as e:
            print(f"Error during idea generation: {str(e)}")
        finally:
            self.result_saver.save_progress()
        return all_ideas

    async def _process_batch(self, batch: List[str]) -> List[Dict]:
        all_batch_ideas = []
        for content in batch:
            try:
                ideas = await self._generate_single_idea(content)
                self.result_saver.add_result(content, ideas)
                all_batch_ideas.append(ideas)
            except Exception as e:
                print(f"Error processing content: {str(e)}")
                all_batch_ideas.append([])
        return all_batch_ideas

    async def _generate_single_idea(self, content: str) -> List[Dict]:
        messages = [
            {
                "role": "system",
                "content": self.custom_prompt
            },
            {
                "role": "user",
                "content": f"Topic: {self.topic}\n\nContent to inspire business ideas:\n{content}"
            }
        ]

        try:
            response = await self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7,
                max_tokens=2000
            )
            
            response_text = response.choices[0].message.content
            
            try:
                ideas = json.loads(response_text)
                if not isinstance(ideas, list):
                    ideas = [ideas]
                return ideas
            except json.JSONDecodeError:
                print(f"Invalid JSON response: {response_text[:100]}...")
                return []
                
        except Exception as e:
            print(f"Error generating ideas: {str(e)}")
            return []

def generate_ideas(
    content_list: List[str],
    topic: str = "web browsing automation",
    custom_prompt: str = None,
    batch_size: int = 5
) -> List[List[Dict]]:
    async def async_generate():
        generator = BusinessIdeaGenerator(
            batch_size=batch_size,
            topic=topic,
            custom_prompt=custom_prompt
        )
        return await generator.generate_ideas(content_list)

    loop = asyncio.get_event_loop()
    return loop.run_until_complete(async_generate())

# Example usage
if __name__ == "__main__":
    
    try:
        ideas = generate_ideas(
            content_list=aggregated_posts,
            topic="web browsing automation"
        )
        
        print("\nProcessing complete!")
        print(f"Results saved to: business_ideas_{datetime.now().strftime('%Y%m%d_%H%M')}.csv")
        
    except KeyboardInterrupt:
        print("\nProcess interrupted by user")
    except Exception as e:
        print(f"Process failed: {str(e)}")

Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_ideas_20241026_1937.csv
Progress saved to business_idea

In [6]:
df = pd.read_csv("business_ideas_20241026_1937.csv")
df.head(2)

,timestamp,original_content,title,business,automation,notes
0,2024-10-26 19:37:20,"30 years of ""freedom"" in South Africa 🇿🇦 - and...",web browsing automation,Automated water resource management and commun...,"[""Step 1: Use Playwright to automatically gath...","{""monetization"": ""Subscription-based model wit..."
1,2024-10-26 19:37:22,Animal Crossing: New Horizons Slipcover\n\nCOM...,Animal Crossing: New Horizons Slipcover,Automated custom slipcover design and printing...,"[""User uploads a game cover image or selects a...","{""monetization"": ""Subscription model with tier..."


In [7]:
df['business'].to_list()[:3]

['Automated water resource management and community alert system',
 'Automated custom slipcover design and printing service for video games',
 'Automated compatibility and update management for Discord plugins and themes']

In [9]:
# First, make sure you have the required packages
# !pip install ipywidgets matplotlib numpy pandas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
%matplotlib inline

def analyze_business(business_name, mean=25000, std_dev=8000, business_list=None):
    """
    Run Monte Carlo simulation for a selected business
    """
    # Generate simulation data
    data = np.random.normal(mean, std_dev, 10000)
    confidence_interval = np.percentile(data, [2.5, 97.5])
    
    # Create plot
    plt.figure(figsize=(12, 6))
    plt.hist(data, bins=30, color='#0066cc', alpha=0.7)
    plt.axvline(mean, color='red', linestyle='--', label='Mean')
    plt.title(f'Monthly Profit Distribution - {business_name}')
    plt.xlabel('Profit ($)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()
    
    # Print statistics
    print("\nKey Statistics:")
    print(f"Mean Monthly Profit: ${mean:,}")
    print(f"Standard Deviation: ${std_dev:,}")
    print(f"95% Confidence Interval: ${int(confidence_interval[0]):,} - ${int(confidence_interval[1]):,}")

# Create the interactive widget
def create_interactive_analysis(business_list):
    interact(
        analyze_business,
        business_name=business_list,
        mean=(0, 100000, 1000),
        std_dev=(1000, 50000, 1000),
        business_list=fixed(business_list)
    )

# Run this next
business_list = df['business'].tolist()
create_interactive_analysis(business_list)

interactive(children=(Dropdown(description='business_name', options=('Automated water resource management and …

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

class BusinessAnalyzer:
    def __init__(self, business_list):
        self.business_list = business_list
        self.setup_widgets()
        
    def setup_widgets(self):
        # Create widgets
        self.business_dropdown = widgets.Dropdown(
            options=self.business_list,
            description='Business:',
            style={'description_width': 'initial'}
        )
        
        self.mean_slider = widgets.IntSlider(
            value=25000,
            min=0,
            max=100000,
            step=1000,
            description='Expected Monthly Profit ($):',
            style={'description_width': 'initial'}
        )
        
        self.std_slider = widgets.IntSlider(
            value=8000,
            min=1000,
            max=50000,
            step=1000,
            description='Standard Deviation ($):',
            style={'description_width': 'initial'}
        )
        
        # Create update button
        self.update_button = widgets.Button(
            description='Update Analysis',
            button_style='primary'
        )
        self.update_button.on_click(self.update_plot)
        
        # Layout widgets
        self.output = widgets.Output()
        
    def generate_histogram_data(self):
        mean = self.mean_slider.value
        std_dev = self.std_slider.value
        return np.random.normal(mean, std_dev, 10000)
        
    def update_plot(self, _):
        with self.output:
            clear_output(wait=True)
            
            # Generate data
            data = self.generate_histogram_data()
            confidence_interval = np.percentile(data, [2.5, 97.5])
            
            # Create figure
            fig, ax = plt.subplots(figsize=(12, 6))
            
            # Plot histogram
            n, bins, patches = ax.hist(data, bins=30, color='#0066cc', alpha=0.7)
            
            # Add mean line
            ax.axvline(self.mean_slider.value, color='red', linestyle='--', label='Mean')
            
            # Customize plot
            ax.set_title(f'Monthly Profit Distribution - {self.business_dropdown.value}')
            ax.set_xlabel('Profit ($)')
            ax.set_ylabel('Frequency')
            ax.legend()
            
            plt.show()
            
            # Display statistics
            print("\nKey Statistics:")
            print(f"Mean Monthly Profit: ${self.mean_slider.value:,}")
            print(f"Standard Deviation: ${self.std_slider.value:,}")
            print(f"95% Confidence Interval: ${int(confidence_interval[0]):,} - ${int(confidence_interval[1]):,}")
    
    def display(self):
        # Display all widgets
        display(widgets.VBox([
            self.business_dropdown,
            self.mean_slider,
            self.std_slider,
            self.update_button,
            self.output
        ]))
        
        # Initial plot
        self.update_plot(None)

# Usage function
def analyze_businesses(business_list):
    analyzer = BusinessAnalyzer(business_list)
    analyzer.display()

# Assuming you have your DataFrame 'df' with a 'business' column
business_list = df['business'].tolist()

# Run the analysis
analyze_businesses(business_list)